In [2]:
import scipy.io as sio

In [5]:
matfile = '/run/media/nfarrugi/ECHACHACHOU/PD_phase/data_probands_beta_noselec_p50_Cz_6_Hz.mat'

In [32]:
filecontents = sio.whosmat(matfile)
print(filecontents)

[('plv_subj_01k1', (1, 1), 'struct'), ('plv_subj_02k1', (1, 1), 'struct'), ('plv_subj_03k1', (1, 1), 'struct'), ('plv_subj_04k1', (1, 1), 'struct'), ('plv_subj_05k1', (1, 1), 'struct'), ('plv_subj_06k1', (1, 1), 'struct'), ('plv_subj_07k1', (1, 1), 'struct'), ('plv_subj_08k1', (1, 1), 'struct'), ('plv_subj_09k1', (1, 1), 'struct'), ('plv_subj_10k1', (1, 1), 'struct'), ('plv_subj_11k1', (1, 1), 'struct'), ('plv_subj_12k1', (1, 1), 'struct'), ('plv_subj_13k1', (1, 1), 'struct'), ('plv_subj_14k1', (1, 1), 'struct'), ('plv_subj_15k1', (1, 1), 'struct'), ('plv_subj_16k1', (1, 1), 'struct'), ('plv_subj_17k1', (1, 1), 'struct'), ('plv_subj_18k1', (1, 1), 'struct'), ('plv_subj_19k1', (1, 1), 'struct'), ('plv_subj_20k1', (1, 1), 'struct')]


Let's pick one subject randomly

In [34]:
import numpy as np 

cursubjname = filecontents[np.random.randint(20)][0]
print(cursubjname)

plv_subj_03k1


Load this subject

In [35]:
subjdata = sio.loadmat(matfile,variable_names=cursubjname,struct_as_record = False,squeeze_me=True)[cursubjname]

We can now directly access all the fields of the struct directly like in Matlab

Let's first select a particular time point to start with, say, 50 ms

In [55]:
subjdata.all.iso.standard.time[125]

0.049999999998811967

Seems to correspond to time point 125. Now let's create an array with all phases at this time point, at Cz, for one condition (Iso std) 

In [62]:
subjdata.all.iso.standard.label

array(['Fz', 'Cz', 'Pz'], dtype=object)

Cz is label 1 

In [64]:
p50_cz_iso_std = subjdata.all.iso.standard.ang[:,1,:,125]

In [65]:
p50_cz_iso_std.shape

(243, 51)

Trials are n_samples, and n_features is the number of frequencies. 

Do the same for RND std

In [68]:
p50_cz_rnd_std = subjdata.all.rnd.standard.ang[:,1,:,125]
p50_cz_rnd_std.shape

(233, 51)

Build the y vector (will be 1 for iso and 2 for rnd)

In [78]:
n_iso = p50_cz_iso_std.shape[0]
n_rnd = p50_cz_rnd_std.shape[0]
print(n_iso,n_rnd)

y = 2*np.ones(n_iso+n_rnd)
y[0:n_iso]=1

243 233


Build the X vector by just concatenating vertically the previous ones 

In [79]:
X = np.vstack([p50_cz_iso_std,p50_cz_rnd_std])
print(X.shape)

(476, 51)